## Inverse Design usign Gradient Opt
---

We loaded test-set (2500 sample), the same one as we load to evaluate the forward model. In which, we define with variable name `df_test`. 

However, in this tutorial we store the first 100 samples in variable name `df_test_100`, and run the optimization only on these samples. Therefore, in order to produce results likely similar as in our paper. We recommed to use `df_test` (2500 samples) in loop process. 

`for idx, row in df_test.iterrows():`

## Import modules

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time
import pymiecs
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
import torch
from sklearn.preprocessing import MinMaxScaler
from wgangp_model import (
    WGAN_GP,
    load_generator,
    generate_synthetic_data,
    inverse_transform_synthetic_data,
)

2025-02-27 15:16:42.934061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 15:16:42.942540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 15:16:42.951212: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 15:16:42.953807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 15:16:42.960754: I tensorflow/core/platform/cpu_feature_guar

## Set dynamic GPU memory growth

In [2]:
# Check if GPU available
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and set to memory growth mode.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected by TensorFlow.")

GPU is available and set to memory growth mode.


I0000 00:00:1740665821.463907   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740665821.466362   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740665821.467648   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


### Define the own resblock class

keras requires custom classes to be defined for being able to reload

In [3]:
# decorator to register the custom resblock to allow serialziation and re-loading
@keras.utils.register_keras_serializable()  # for keras3
class ResBlock1D(keras.Model):
    def __init__(self, filters, kernel_size=3, convblock=False, **kwargs):
        super(ResBlock1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

        # setup all necessary layers
        self.conv1 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn1 = keras.layers.BatchNormalization()

        self.conv2 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn2 = keras.layers.BatchNormalization()

        # self.relu = keras.layers.LeakyReLU()
        self.relu = keras.layers.LeakyReLU(negative_slope=0.01)

        self.convblock = convblock
        if self.convblock:
            self.conv_shortcut = keras.layers.Conv1D(filters, 1)

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        # add shortcut. optionally pass it through a Conv
        if self.convblock:
            x_sc = self.conv_shortcut(input_tensor)
        else:
            x_sc = input_tensor
        x += x_sc
        return self.relu(x)

    def get_config(self):
        base_config = super().get_config()
        return {
            "convblock": self.convblock,
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            **base_config,
        }

## Reload the forward and wgangp model

In [5]:
forward_path = "models/resnet_Mie_predictor.keras"
wgangp_path = "models/wgangp_generator.h5"

forward_model = keras.models.load_model(forward_path)
generator = load_generator(wgangp_path)

I0000 00:00:1740665848.299190   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740665848.301630   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740665848.302861   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740665848.444976   14756 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

## Relaod the data scaler

In [6]:
# Load the preprocessors and scalers
preprocessor_path = "datasets/scaler_particle_geometries.pkl"
scaler_Qfwd_path = "datasets/scaler_Qfwd.pkl"
scaler_Qback_path = "datasets/scaler_Qback.pkl"

# Load the preprocessors and scalers
with open(preprocessor_path, "rb") as f:
    preprocessor = pickle.load(f)
with open(scaler_Qfwd_path, "rb") as f:
    scaler_Qfwd = pickle.load(f)
with open(scaler_Qback_path, "rb") as f:
    scaler_Qback = pickle.load(f)

## Reload Test data 

In [7]:
hdf5_df_file = "datasets/core_shell_particles_raw_122500_test_with_pred.h5"
df_test = pd.read_hdf(hdf5_df_file)
df_test.head()  # 2500 samples

,mat_core,mat_shell,r_core,r_shell,wavelength,Q_sca,Q_back,Q_fwd,Q_fwd_pred,Q_back_pred
0,ZrO2,Si,39,132,"[400.0, 406.3492063492063, 412.6984126984127, ...","[2.038010369711505, 1.8095124021006057, 1.4807...","[0.6439440129016081, 0.44670942047839124, 0.55...","[8.631606079996017, 7.795760005483267, 6.22831...","[8.785859, 8.286023, 6.1634374, 4.3949394, 4.2...","[0.81082445, 0.509972, 0.4711062, 0.62237597, ..."
1,Au,Si3N4,65,92,"[400.0, 406.3492063492063, 412.6984126984127, ...","[0.7478421540587286, 0.7347653970480106, 0.723...","[0.25254436594760143, 0.2641361829295818, 0.27...","[3.0595320722390023, 2.910348641419738, 2.7718...","[2.9980655, 2.9284086, 2.8653688, 2.730851, 2....","[0.33432025, 0.30205694, 0.30620533, 0.3165547..."
2,Si,Si,31,32,"[400.0, 406.3492063492063, 412.6984126984127, ...","[0.42059673950184884, 0.2840222012565064, 0.21...","[0.12245274773436957, 0.03833342713719667, 0.0...","[1.1619876883360172, 0.8312774845838197, 0.618...","[0.92039216, 1.3765805, 1.5048053, 1.2357119, ...","[1.2127657, 0.83550537, 0.32235485, 0.0964607,..."
3,ZrO2,ZrO2,81,114,"[400.0, 406.3492063492063, 412.6984126984127, ...","[4.515870402326752, 4.215809011163139, 4.05102...","[2.008253901402069, 1.6064395105460934, 1.4057...","[19.18042348996285, 16.158873919994733, 14.280...","[18.862698, 16.10313, 14.292469, 12.995516, 11...","[1.9835316, 1.5604014, 1.3563392, 1.3436563, 1..."
4,Au,TiO2,49,76,"[400.0, 406.3492063492063, 412.6984126984127, ...","[2.5633444541178934, 2.5892776814208465, 2.588...","[1.6053755830270218, 1.4718419539005771, 1.314...","[6.575478822430851, 6.834712459103053, 7.04137...","[6.3441553, 6.6466975, 7.2830606, 7.7183237, 7...","[2.265903, 2.1518085, 2.0174856, 1.8726242, 1...."


In [8]:
df_test["log_Qfwd"] = df_test["Q_fwd"].apply(lambda x: np.log1p(np.array(x)))
df_test["log_Qback"] = df_test["Q_back"].apply(lambda x: np.log1p(np.array(x)))

In [ ]:
df_test_100 = df_test.head(100)

## Mie Utils
---

In [18]:
# %% --- Mie
def get_Mie_spec(wavelengths, r_core, r_shell, mat_core, mat_shell, n_env):

    k0 = 2 * np.pi / wavelengths
    n_core = mat_core.get_refindex(wavelengths)
    n_shell = mat_shell.get_refindex(wavelengths)

    res = pymiecs.Q(
        k0,
        r_core=r_core,
        n_core=n_core,
        r_shell=r_shell,
        n_shell=n_shell,
        n_env=n_env.real**0.5,  # host medium must be lossless
    )
    return (
        res["qsca"],
        res["qback"],
        res["qfwd"],
    )

In [19]:
Si = pymiecs.materials.MaterialDatabase("Si")
SiO2 = pymiecs.materials.MaterialDatabase("SiO2")
Si3N4 = pymiecs.materials.MaterialDatabase("Si3N4")
Au = pymiecs.materials.MaterialDatabase("Au")
Ag = pymiecs.materials.MaterialDatabase("Ag")
ZrO2 = pymiecs.materials.MaterialDatabase("ZrO2")
TiO2 = pymiecs.materials.MaterialDatabase("TiO2")


# Define a function to map material names to material objects
def get_material(material_name):
    if material_name == "Si":
        return Si
    elif material_name == "SiO2":
        return SiO2
    elif material_name == "Au":
        return Au
    elif material_name == "Ag":
        return Ag
    elif material_name == "Si3N4":
        return Si3N4
    elif material_name == "ZrO2":
        return ZrO2
    elif material_name == "TiO2":
        return TiO2
    else:
        raise ValueError(f"Unknown material: {material_name}")

## Adam with Batch Processing

In [20]:
batch_size = 500  # Batch size for parallel optimization
latent_dim = 128  # Dimension of the latent space
optimized_geometries_list = []

In [21]:
def objective_function_batch(
    z_batch, generator, forward_model, target_Qfwd_transformed, target_Qback_transformed
):
    synthetic_geometries = generator(z_batch)
    synthetic_geometries_concat = keras.ops.concatenate(synthetic_geometries, axis=1)

    predicted_batch = forward_model(
        synthetic_geometries_concat
    )  # Forward pass through the forward model

    # Calculate MSE loss for Qfwd and Qback across all vectors in the batch
    loss_Qfwd_batch = keras.ops.mean(
        keras.ops.square(predicted_batch[..., 0] - target_Qfwd_transformed), axis=1
    )
    loss_Qback_batch = keras.ops.mean(
        keras.ops.square(predicted_batch[..., 1] - target_Qback_transformed), axis=1
    )

    # Total loss for the batch (sum of both losses)
    total_loss_batch = loss_Qfwd_batch + loss_Qback_batch
    return total_loss_batch

In [22]:
def learning_rate_schedule(iteration, initial_lr=0.1):
    if iteration < 50:
        return initial_lr
    else:
        return initial_lr * 0.1

In [23]:
def optimize_latent_vector_parallel(
    z_batch,
    generator,
    forward_model,
    target_Qfwd_transformed,
    target_Qback_transformed,
    initial_lr=0.01,
    iterations=250,
    use_lr_schedule=False,
):
    # Initialize the optimizer with the initial learning rate
    if not isinstance(z_batch, tf.Variable):
        raise ValueError("z_batch must be a tf.Variable.")

    optimizer = keras.optimizers.Adam(learning_rate=initial_lr)
    loss_history = []

    for i in range(iterations):
        # Update the learning rate if using learning rate schedule
        if use_lr_schedule:
            new_learning_rate = learning_rate_schedule(i, initial_lr)
            optimizer.learning_rate.assign(new_learning_rate)

        with tf.GradientTape() as tape:
            tape.watch(z_batch)
            # Calculate the loss for the batch of latent vectors
            total_loss_batch = objective_function_batch(
                z_batch,
                generator,
                forward_model,
                target_Qfwd_transformed,
                target_Qback_transformed,
            )

        # Get gradients and update latent vectors in the batch
        gradients_batch = tape.gradient(total_loss_batch, [z_batch])
        optimizer.apply_gradients(zip(gradients_batch, [z_batch]))
        loss_history.append(total_loss_batch.numpy())

    return z_batch, loss_history, total_loss_batch

In [15]:
def process_and_optimize_sample(
    sample_row,
    generator,
    forward_model,
    scaler_Qfwd,
    scaler_Qback,
    batch_size,
    initial_lr=0.01,
    iterations=250,
    use_lr_schedule=False,
):
    # Extract and transform Qfwd and Qback for the sample
    target_Qfwd = np.array(sample_row["log_Qfwd"]).reshape(1, -1)
    target_Qback = np.array(sample_row["log_Qback"]).reshape(1, -1)

    # Apply MinMax scaling using the preloaded scalers
    target_Qfwd_transformed = scaler_Qfwd.transform(target_Qfwd)
    target_Qback_transformed = scaler_Qback.transform(target_Qback)

    # Initialize batch of latent vectors
    initial_z_batch = np.random.normal(size=(batch_size, latent_dim))
    z_batch_tf = tf.Variable(initial_z_batch, dtype=tf.float32)

    # Adam-based batch optimization function
    optimized_z_batch, loss_history, total_loss_batch = optimize_latent_vector_parallel(
        z_batch_tf,
        generator,
        forward_model,
        target_Qfwd_transformed,
        target_Qback_transformed,
        initial_lr=initial_lr,
        iterations=iterations,
        use_lr_schedule=use_lr_schedule,
    )

    # Return optimized latent vectors and corresponding losses
    return (
        optimized_z_batch,
        loss_history,
        total_loss_batch.numpy(),
    )

## Process 2500 Samples

In [24]:
# Initialize a list to store the runtime for each sample
num_samples = 2500
sample_runtimes = []
optimized_geometries_list = []

for i, sample_row in df_test_100.iterrows():
    print(f"Processing sample {i + 1}/{num_samples}")
    start_time = time.time()  # Start the timer

    # Optimize the latent vectors for the current sample
    optimized_latent_vectors, loss_history, total_loss_batch = (
        process_and_optimize_sample(
            sample_row,
            generator,
            forward_model,
            scaler_Qfwd,
            scaler_Qback,
            batch_size=batch_size,
            initial_lr=0.01,
            iterations=100,
            use_lr_schedule=False,  # set to false because we don't use leanring rate schedule
        )
    )
    # Record end time and calculate sample runtime
    end_time = time.time()
    sample_runtime = end_time - start_time

    # Append runtime to the list
    sample_runtimes.append({"Sample": i + 1, "Runtime (seconds)": sample_runtime})
    print(f"Sample {i + 1} runtime: {sample_runtime:.2f} seconds")
    print("---------------------------------------------")
    optimized_geometries_list.append((optimized_latent_vectors, total_loss_batch))
"""
##########################################################################################
####################### Save the runtimes into a DataFrame ###############################
##########################################################################################
"""
runtime_df = pd.DataFrame(sample_runtimes)
sample_runtime_path = "runtime/gradient_inverse_runtime.pkl"

# Saving DataFrames
with open(sample_runtime_path, "wb") as f:
    pickle.dump(runtime_df, f)

Processing sample 1/2500
Sample 1 runtime: 9.19 seconds
---------------------------------------------
Processing sample 2/2500
Sample 2 runtime: 9.13 seconds
---------------------------------------------
Processing sample 3/2500
Sample 3 runtime: 9.08 seconds
---------------------------------------------
Processing sample 4/2500
Sample 4 runtime: 8.98 seconds
---------------------------------------------
Processing sample 5/2500
Sample 5 runtime: 9.22 seconds
---------------------------------------------
Processing sample 6/2500
Sample 6 runtime: 9.12 seconds
---------------------------------------------
Processing sample 7/2500
Sample 7 runtime: 9.11 seconds
---------------------------------------------
Processing sample 8/2500
Sample 8 runtime: 9.19 seconds
---------------------------------------------
Processing sample 9/2500
Sample 9 runtime: 9.07 seconds
---------------------------------------------
Processing sample 10/2500
Sample 10 runtime: 8.92 seconds
------------------------

## Evaluate the Optimization

In [25]:
# Initialize a list to hold the DataFrames for each sample
optimized_samples_list = []
best_geometries_list = []

for i, (optimized_latent_vectors, total_loss_batch) in enumerate(
    optimized_geometries_list
):
    print(f"Generating optimized geometries for sample {i + 1}")

    # Generate the synthetic geometries using the WGAN-GP generator
    synthetic_geometries = generator(optimized_latent_vectors)
    synthetic_geometries_concat = tf.concat(synthetic_geometries, axis=1)

    # Perform the inverse transformation to convert back to the original scale
    inverse_synthetic_data = inverse_transform_synthetic_data(
        preprocessor, synthetic_geometries
    )

    # Create the DataFrame for the optimized geometries
    desired_columns = ["mat_core", "mat_shell", "r_core", "r_shell"]
    generated_columns = ["r_core", "r_shell", "mat_core", "mat_shell"]

    optimized_geometries_df = pd.DataFrame(
        inverse_synthetic_data, columns=generated_columns
    )
    optimized_geometries_df = optimized_geometries_df[desired_columns]

    optimized_geometries_df["Total Loss"] = total_loss_batch
    optimized_geometries_df["Sample"] = i + 1

    # Sort the DataFrame by the 'Total Loss' column (ascending order: best to worst)
    optimized_geometries_df_sorted = optimized_geometries_df.sort_values(
        by="Total Loss", ascending=True
    )
    best_geometries = pd.DataFrame(optimized_geometries_df_sorted.iloc[[0]])

    # Append the DataFrame to the list
    optimized_samples_list.append(optimized_geometries_df)
    best_geometries_list.append(best_geometries)

# Combine all the DataFrames into one large DataFrame:
final_optimized_df = pd.concat(optimized_samples_list, ignore_index=True)
final_best_geometries_df = pd.concat(best_geometries_list, ignore_index=True)

final_best_geometries_df

Generating optimized geometries for sample 1
Generating optimized geometries for sample 2
Generating optimized geometries for sample 3
Generating optimized geometries for sample 4
Generating optimized geometries for sample 5
Generating optimized geometries for sample 6
Generating optimized geometries for sample 7
Generating optimized geometries for sample 8
Generating optimized geometries for sample 9
Generating optimized geometries for sample 10
Generating optimized geometries for sample 11
Generating optimized geometries for sample 12
Generating optimized geometries for sample 13
Generating optimized geometries for sample 14
Generating optimized geometries for sample 15
Generating optimized geometries for sample 16
Generating optimized geometries for sample 17
Generating optimized geometries for sample 18
Generating optimized geometries for sample 19
Generating optimized geometries for sample 20
Generating optimized geometries for sample 21
Generating optimized geometries for sample 

,mat_core,mat_shell,r_core,r_shell,Total Loss,Sample
0,ZrO2,Si,43.290783,134.534927,0.008879,1
1,Au,Si3N4,64.626152,92.342316,0.000759,2
2,Au,Si,9.229283,29.025507,0.000859,3
3,ZrO2,ZrO2,36.042091,112.510147,0.000520,4
4,Au,TiO2,49.659683,78.127953,0.001434,5
...,...,...,...,...,...,...
95,SiO2,SiO2,41.110489,109.47319,0.000340,96
96,SiO2,SiO2,77.914291,137.589569,0.000565,97
97,Si,Ag,64.466721,98.040482,0.004894,98
98,Ag,Au,63.86573,146.184692,0.000400,99


### Forward Model predict on optimized geometries

In [26]:
categorical_features = ["mat_core", "mat_shell"]
numerical_features = ["r_core", "r_shell"]

# Use the preprocessor to transform the data
X_optimized_preprocessed = preprocessor.transform(
    final_best_geometries_df[categorical_features + numerical_features]
)
# Predict using the forward model
y_pred = forward_model.predict(X_optimized_preprocessed)

# Separate predictions for Qfwd and Qback
y_pred_Qfwd = y_pred[..., 0]
y_pred_Qback = y_pred[..., 1]

# Inverse transform the predictions to their original scale
y_pred_Qfwd_inverse = scaler_Qfwd.inverse_transform(y_pred_Qfwd)
y_pred_Qback_inverse = scaler_Qback.inverse_transform(y_pred_Qback)

# Apply expm1 to revert the log1p transformation
y_pred_Qfwd_orig_scale = np.expm1(y_pred_Qfwd_inverse)
y_pred_Qback_orig_scale = np.expm1(y_pred_Qback_inverse)

# Add the predicted values (in original scale) to the DataFrame
final_best_geometries_df["predicted_Qfwd"] = y_pred_Qfwd_orig_scale.tolist()
final_best_geometries_df["predicted_Qback"] = y_pred_Qback_orig_scale.tolist()

# Display the updated DataFrame with predictions
print("Predicted Qfwd (Original Scale):", y_pred_Qfwd_orig_scale)
print("Predicted Qback (Original Scale):", y_pred_Qback_orig_scale)
final_best_geometries_df

I0000 00:00:1740667010.140820   23014 service.cc:146] XLA service 0x72770c0067d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740667010.140840   23014 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-27 15:36:50.165350: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


1/4 ━━━━━━━━━━━━━━━━━━━━ 2s 914ms/step

I0000 00:00:1740667010.599006   23014 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 328ms/step
Predicted Qfwd (Original Scale): [[ 9.1920996e+00  8.3832588e+00  5.9863801e+00 ...  5.1359053e+00
   5.0155325e+00  4.9526200e+00]
 [ 3.0474691e+00  2.9724171e+00  2.9058237e+00 ...  2.3839619e+00
   2.3039286e+00  2.1644197e+00]
 [ 7.6678884e-01  7.1648824e-01  6.2262148e-01 ...  2.3297865e-02
   2.9154822e-02 -2.9959276e-04]
 ...
 [ 1.4088315e+01  1.2640507e+01  1.1228907e+01 ...  1.4902735e+00
   1.4185132e+00  1.3727887e+00]
 [ 1.3098956e+01  1.2716630e+01  1.2298725e+01 ...  4.7805219e+00
   4.7856750e+00  4.7881708e+00]
 [ 8.0186882e+00  8.0810442e+00  7.5959845e+00 ...  2.9283466e+00
   3.2158878e+00  3.3312092e+00]]
Predicted Qback (Original Scale): [[ 8.3439088e-01  5.4236066e-01  4.7147119e-01 ...  6.8850012e+00
   6.8896651e+00  7.0829792e+00]
 [ 3.3788413e-01  3.0526477e-01  3.0672398e-01 ...  2.1848106e+00
   2.1439345e+00  2.0472090e+00]
 [ 3.0661443e-01  1.8447050e-01  5.4303367e-02 ... -5.2876882e-03
  -7.9232333e-03 -2.3213762e-0

,mat_core,mat_shell,r_core,r_shell,Total Loss,Sample,predicted_Qfwd,predicted_Qback
0,ZrO2,Si,43.290783,134.534927,0.008879,1,"[9.192099571228027, 8.383258819580078, 5.98638...","[0.8343908786773682, 0.5423606634140015, 0.471..."
1,Au,Si3N4,64.626152,92.342316,0.000759,2,"[3.047469139099121, 2.972417116165161, 2.90582...","[0.3378841280937195, 0.30526477098464966, 0.30..."
2,Au,Si,9.229283,29.025507,0.000859,3,"[0.7667888402938843, 0.716488242149353, 0.6226...","[0.3066144287586212, 0.1844705045223236, 0.054..."
3,ZrO2,ZrO2,36.042091,112.510147,0.000520,4,"[16.881980895996094, 14.792258262634277, 13.37...","[1.830836534500122, 1.4777687788009644, 1.3052..."
4,Au,TiO2,49.659683,78.127953,0.001434,5,"[5.846821308135986, 6.094841480255127, 6.67513...","[2.66878342628479, 2.577922821044922, 2.472384..."
...,...,...,...,...,...,...,...,...
95,SiO2,SiO2,41.110489,109.47319,0.000340,96,"[4.2617292404174805, 4.091280937194824, 3.8542...","[0.06856991350650787, 0.06565897166728973, 0.0..."
96,SiO2,SiO2,77.914291,137.589569,0.000565,97,"[9.943931579589844, 9.73583984375, 9.08984375,...","[0.2530745565891266, 0.27845463156700134, 0.28..."
97,Si,Ag,64.466721,98.040482,0.004894,98,"[14.0883150100708, 12.640506744384766, 11.2289...","[0.3211176097393036, 0.29605036973953247, 0.38..."
98,Ag,Au,63.86573,146.184692,0.000400,99,"[13.098956108093262, 12.716629981994629, 12.29...","[0.7737025618553162, 0.8242150545120239, 0.920..."


### Use Mie to calculate Mie Qfwd and Qback on optimized geometries

In [27]:
wavelengths = np.linspace(400, 800, 64)  # From 400 nm to 800 nm
n_env = 1.0
mie_Qfwd_list, mie_Qback_list = [], []

for idx, row in final_best_geometries_df.iterrows():
    mat_core = get_material(row["mat_core"])
    mat_shell = get_material(row["mat_shell"])
    r_core = row["r_core"]
    r_shell = row["r_shell"]

    _, Qback, Qfwd = get_Mie_spec(
        wavelengths, r_core, r_shell, mat_core, mat_shell, n_env
    )

    mie_Qfwd_list.append(Qfwd)
    mie_Qback_list.append(Qback)

# Convert the Mie Qfwd and Qback to DataFrame or arrays if needed
mie_Qfwd_array = np.array(mie_Qfwd_list)
mie_Qback_array = np.array(mie_Qback_list)

# Validation: Check dimensions
assert len(mie_Qfwd_array) == len(
    final_best_geometries_df
), "Mismatch in dimensions for Qfwd!"
assert len(mie_Qback_array) == len(
    final_best_geometries_df
), "Mismatch in dimensions for Qback!"

# Add the Mie results to the DataFrame
final_best_geometries_df["mie_Qfwd"] = mie_Qfwd_array.tolist()
final_best_geometries_df["mie_Qback"] = mie_Qback_array.tolist()

final_best_geometries_df

,mat_core,mat_shell,r_core,r_shell,Total Loss,Sample,predicted_Qfwd,predicted_Qback,mie_Qfwd,mie_Qback
0,ZrO2,Si,43.290783,134.534927,0.008879,1,"[9.192099571228027, 8.383258819580078, 5.98638...","[0.8343908786773682, 0.5423606634140015, 0.471...","[8.746778103170982, 7.91586330968485, 6.211498...","[0.7775276166180702, 0.4971269442263212, 0.557..."
1,Au,Si3N4,64.626152,92.342316,0.000759,2,"[3.047469139099121, 2.972417116165161, 2.90582...","[0.3378841280937195, 0.30526477098464966, 0.30...","[3.1341925340839234, 2.976245836421121, 2.8294...","[0.2559239767541835, 0.2646544273424956, 0.275..."
2,Au,Si,9.229283,29.025507,0.000859,3,"[0.7667888402938843, 0.716488242149353, 0.6226...","[0.3066144287586212, 0.1844705045223236, 0.054...","[0.43872836499497514, 0.3478222590641083, 0.28...","[0.04005127016786281, 0.049663206733242794, 0...."
3,ZrO2,ZrO2,36.042091,112.510147,0.000520,4,"[16.881980895996094, 14.792258262634277, 13.37...","[1.830836534500122, 1.4777687788009644, 1.3052...","[16.70977580206304, 14.58948653434555, 13.2694...","[1.6910900029907083, 1.447648287585261, 1.3376..."
4,Au,TiO2,49.659683,78.127953,0.001434,5,"[5.846821308135986, 6.094841480255127, 6.67513...","[2.66878342628479, 2.577922821044922, 2.472384...","[6.014242055296438, 6.3195079097322795, 6.6321...","[1.94730550246385, 1.8670510218373575, 1.76482..."
...,...,...,...,...,...,...,...,...,...,...
95,SiO2,SiO2,41.110489,109.47319,0.000340,96,"[4.2617292404174805, 4.091280937194824, 3.8542...","[0.06856991350650787, 0.06565897166728973, 0.0...","[4.573288387162043, 4.245011666314171, 3.94391...","[0.03946909197739819, 0.042226609888916924, 0...."
96,SiO2,SiO2,77.914291,137.589569,0.000565,97,"[9.943931579589844, 9.73583984375, 9.08984375,...","[0.2530745565891266, 0.27845463156700134, 0.28...","[10.574533160291782, 10.072956082619692, 9.599...","[0.4070526865165429, 0.37432090605714285, 0.33..."
97,Si,Ag,64.466721,98.040482,0.004894,98,"[14.0883150100708, 12.640506744384766, 11.2289...","[0.3211176097393036, 0.29605036973953247, 0.38...","[16.7264238967491, 15.130698836324552, 12.3324...","[0.14089732119645054, 0.1969627064222528, 0.31..."
98,Ag,Au,63.86573,146.184692,0.000400,99,"[13.098956108093262, 12.716629981994629, 12.29...","[0.7737025618553162, 0.8242150545120239, 0.920...","[13.20370222712471, 12.843223017398184, 12.507...","[0.857515461892484, 0.9033743389330479, 0.9350..."


In [28]:
final_best_geometries_df["log_Qfwd"] = final_best_geometries_df["mie_Qfwd"].apply(
    lambda x: np.log1p(np.array(x))
)
final_best_geometries_df["log_Qback"] = final_best_geometries_df["mie_Qback"].apply(
    lambda x: np.log1p(np.array(x))
)
final_best_geometries_df.head()

,mat_core,mat_shell,r_core,r_shell,Total Loss,Sample,predicted_Qfwd,predicted_Qback,mie_Qfwd,mie_Qback,log_Qfwd,log_Qback
0,ZrO2,Si,43.290783,134.534927,0.008879,1,"[9.192099571228027, 8.383258819580078, 5.98638...","[0.8343908786773682, 0.5423606634140015, 0.471...","[8.746778103170982, 7.91586330968485, 6.211498...","[0.7775276166180702, 0.4971269442263212, 0.557...","[2.276936779441985, 2.187832084504897, 1.97567...","[0.5752234193498501, 0.4035479009246313, 0.443..."
1,Au,Si3N4,64.626152,92.342316,0.000759,2,"[3.047469139099121, 2.972417116165161, 2.90582...","[0.3378841280937195, 0.30526477098464966, 0.30...","[3.1341925340839234, 2.976245836421121, 2.8294...","[0.2559239767541835, 0.2646544273424956, 0.275...","[1.419292033490004, 1.3803381169687217, 1.3427...","[0.22787153815215405, 0.23479890489263294, 0.2..."
2,Au,Si,9.229283,29.025507,0.000859,3,"[0.7667888402938843, 0.716488242149353, 0.6226...","[0.3066144287586212, 0.1844705045223236, 0.054...","[0.43872836499497514, 0.3478222590641083, 0.28...","[0.04005127016786281, 0.049663206733242794, 0....","[0.36375964357858215, 0.2984901485027536, 0.25...","[0.0392700101764927, 0.04846935722423655, 0.05..."
3,ZrO2,ZrO2,36.042091,112.510147,0.000520,4,"[16.881980895996094, 14.792258262634277, 13.37...","[1.830836534500122, 1.4777687788009644, 1.3052...","[16.70977580206304, 14.58948653434555, 13.2694...","[1.6910900029907083, 1.447648287585261, 1.3376...","[2.874116792316703, 2.7465967469497183, 2.6581...","[0.989946317112894, 0.8951276809560381, 0.8491..."
4,Au,TiO2,49.659683,78.127953,0.001434,5,"[5.846821308135986, 6.094841480255127, 6.67513...","[2.66878342628479, 2.577922821044922, 2.472384...","[6.014242055296438, 6.3195079097322795, 6.6321...","[1.94730550246385, 1.8670510218373575, 1.76482...","[1.947942661429967, 1.9905431002670786, 2.0323...","[1.0808913639625137, 1.0532839829889078, 1.016..."


## Save best geometries

In [29]:
best_geometries_path = "best_geometries/gradient_inverse_test_data.pkl"

# Saving DataFrames
with open(best_geometries_path, "wb") as f:
    pickle.dump(final_best_geometries_df, f)